This notebook is used to download previously search planet images

In [ ]:
import backoff
import os
from pathlib import Path
import requests
from tqdm.auto import tqdm
from planet import api as planet_api
from time import sleep

from shapely.geometry import shape
from shapely_geojson import dumps
import geopandas as gpd
import pandas as pd
import json

In [ ]:
# Set session auth
PLANET_API_KEY = os.getenv("PLANET_API_KEY")
session = requests.Session()
session.auth = requests.auth.HTTPBasicAuth(PLANET_API_KEY, '')
session.headers.update({'Content-Type': 'application/json; charset=utf-8', 'Accept': 'application/json'})

# Set connection urls
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

In [ ]:
ids_path = Path("0_west_afk/1802_ids.txt")
with open(ids_path) as f:
    item_ids = [l.strip() for l in f.read().split(",")]
    # item_ids = [line.replace("\n","") for line in f]

In [ ]:
fishnet = gpd.GeoDataFrame.from_file("0_west_afk/shp/aoi_fishnet.shp")
fishnet;

In [ ]:
# This dataframe was created by using some geoprocessings in QGIS
# To be sure the entire area was fully covered by images.
all_images_path = "0_west_afk/shp/all_foot_images.shp"
all_images_gdf = gpd.GeoDataFrame.from_file(all_images_path)
images_ids = all_images_gdf.id.unique()

In [ ]:
def extract_geometry(item_id):
    """Creates a geodataframe containing image id and its geometries"""
    
    url=f"https://api.planet.com/data/v1/item-types/PSScene/items/{item_id}"
    response = requests.get(url, auth=session.auth)
    json = response.json()
    
    return {
        "id":json["id"], 
        "date":json["properties"]["acquired"],
        "geometry": shape(json["geometry"])
    }

image_geometries = [extract_geometry(item_id) for item_id in images_ids]

In [ ]:
image_gdf = gpd.GeoDataFrame(image_geometries)

In [ ]:
image_gdf["date"]=pd.to_datetime(image_gdf["date"]) 

In [ ]:
main_date = image_gdf.date.dt.strftime('%d-%m-%y').unique()[0]
image_gdf["date"]=image_gdf.date.dt.strftime('%d-%m-%y')
# image_gdf.to_file(f"0_west_afk/shp/images_geometry_{main_date}.shp", driver="ESRI Shapefile")
main_date

## Get all images associated to each grid 

In [ ]:

fishnet;
all_images_gdf;

In [ ]:
"""Creates a dictionary with grid_id associated with planet id images"""

images_by_grid = {}

for idx, grid in fishnet.iterrows():
    
    images_by_grid[idx] = []
    
    for _, img in all_images_gdf.iterrows():

        if grid.geometry.intersects(img.geometry):

            images_by_grid[idx].append(img.id)

In [ ]:
images_by_grid[0]

In [ ]:
@backoff.on_exception(
    backoff.expo,
    (planet_api.exceptions.OverQuota, planet_api.exceptions.TooManyRequests),
    max_time=360
)
def post_order(grid_id):
    

    order_request = {
        "name": str(grid_id),
        "order_type": "partial",
        "products": [
            {
                "item_type": "PSScene",
                "item_ids": images_by_grid[grid_id],
                "product_bundle": "analytic_8b_sr_udm2"
            }
        ],
        "tools": [
            {"clip": {"aoi": json.loads(dumps(fishnet.iloc[grid_id].geometry))}},
        ],
        "delivery": {
            "google_earth_engine": {
                "project": "planet-afk",
                "collection": "afk_treecount"
            }
        },
        "notifications": {"email": True},
    }
    
    # Send order to server
    return requests.post(
        orders_url, 
        data=json.dumps(order_request), 
        auth=session.auth, 
        headers=session.headers
    )
    

In [ ]:
responses = {}
for grid_id in tqdm(images_by_grid):
    if grid_id<=10: continue
    responses[grid_id] = post_order(grid_id)
    print(grid_id)
    sleep(1)